# Laboratorio 6
 - Marco Ramirez 21032
 - Josué Morales 21116

Dataset:
https://drive.google.com/drive/u/0/folders/1TI91fx9zsxKKPiOyrRyfW8ggG9C0RrRb

In [ ]:
from google.colab import drive
from tensorflow.keras import layers

import os
import concurrent.futures

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Unzipping images into Google Drive folder.

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip /content/drive/MyDrive/CelebA/images.zip -d /content/drive/MyDrive/CelebA/dataset/

### Load the CSV files for partitioning and other metadata.

In [ ]:
# Paths to CSV files
partition_file = '/content/drive/MyDrive/CelebA/list_eval_partition.csv'
bbox_file = '/content/drive/MyDrive/CelebA/list_bbox_celeba.csv'
landmarks_file = '/content/drive/MyDrive/CelebA/list_landmarks_align_celeba.csv'
attributes_file = '/content/drive/MyDrive/CelebA/list_attr_celeba.csv'

# Load data into dataframes
partitions = pd.read_csv(partition_file)
bbox = pd.read_csv(bbox_file)
landmarks = pd.read_csv(landmarks_file)
attributes = pd.read_csv(attributes_file)

### Image Preprocessing: Resizing and Normalization

In [ ]:
# Path to images
image_folder = '/content/drive/MyDrive/CelebA/dataset/img_align_celeba/img_align_celeba/'
# Google Drive folder for saving processed images
processed_images_folder = '/content/drive/MyDrive/CelebA/dataset/processed_images/'

In [ ]:
# Create the folder if it doesn't exist
os.makedirs(processed_images_folder, exist_ok=True)

# Function to preprocess (resize and normalize) and save an image
def preprocess_and_save_image(image_name, image_folder, output_folder, image_size=(128, 128)):
    # Load the image
    image_path = os.path.join(image_folder, image_name)
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)

    # Resize the image to the target size (128x128 by default)
    resized_image = tf.image.resize(image, image_size)

    # Normalize the image to the range [-1, 1]
    normalized_image = (resized_image / 127.5) - 1.0

    # Convert the TensorFlow tensor to a NumPy array
    normalized_image_np = normalized_image.numpy()

    # Save the normalized image as a .npy file
    output_image_path = os.path.join(output_folder, image_name.replace('.jpg', '.npy'))
    np.save(output_image_path, normalized_image_np)

    return output_image_path

# List of all images from 000001.jpg to 202599.jpg
image_filenames = [f'{i:06d}.jpg' for i in range(1, 202600)]

# Function to run in parallel using threads
def process_images_in_parallel(image_filenames, image_folder, processed_images_folder, max_workers=8):
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Submit tasks to the pool and run them in parallel using threads
        futures = [executor.submit(preprocess_and_save_image, image_name, image_folder, processed_images_folder)
                   for image_name in image_filenames]

        # Wait for all tasks to complete and print progress
        for future in concurrent.futures.as_completed(futures):
            try:
                result = future.result()  # Get the result (or exception) from each task
                print(f'Processed and saved as .npy: {result}')
            except Exception as exc:
                print(f'Image processing generated an exception: {exc}')

# Run the preprocessing pipeline with parallelism using threads
process_images_in_parallel(image_filenames, image_folder, processed_images_folder, max_workers=8)

### GAN Implementation

#### Load Normalized Images On-the-Fly with tf.data.Dataset

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

# Function to load a single NumPy array from the .npy file
def load_image(image_id):
    image_id = str(image_id.numpy().decode('utf-8'))  # Convert the EagerTensor to a string
    image_path = os.path.join(processed_images_folder, image_id.replace('.jpg', '.npy'))
    image_array = np.load(image_path)
    return image_array

# Function to wrap NumPy loading in a TensorFlow function
def load_image_wrapper(image_id):
    return tf.py_function(func=load_image, inp=[image_id], Tout=tf.float32)

# Modify this function to reduce the dataset size
def create_tf_dataset_with_loading(partition_df, batch_size, partition_filter, frac=1.0):
    # Filter the dataset based on partition (0 = train, 1 = validation, 2 = test)
    filtered_partition = partition_df[partition_df['partition'] == partition_filter]

    # Subsample a fraction of the dataset if 'frac' is less than 1.0
    if frac < 1.0:
        filtered_partition = filtered_partition.sample(frac=frac)

    # Extract the image IDs
    image_ids = filtered_partition['image_id'].values

    # Create a TensorFlow dataset from image IDs
    dataset = tf.data.Dataset.from_tensor_slices(image_ids)

    # Map the image IDs to loading the corresponding NumPy files on-the-fly
    dataset = dataset.map(lambda image_id: load_image_wrapper(image_id), num_parallel_calls=tf.data.AUTOTUNE)

    # Shuffle, batch, and prefetch the dataset
    dataset = dataset.shuffle(buffer_size=10000)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

    return dataset

# Example: Use only 10% of the training data (frac=0.1)
train_dataset = create_tf_dataset_with_loading(partitions, batch_size=128, partition_filter=0, frac=0.1)
val_dataset = create_tf_dataset_with_loading(partitions, batch_size=128, partition_filter=1)

#### Design of the Generator

In [ ]:
def build_generator(noise_dim):
    model = tf.keras.Sequential()

    # First fully connected layer
    model.add(layers.Dense(8 * 8 * 256, use_bias=False, input_shape=(noise_dim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Reshape to start convolutional stack
    model.add(layers.Reshape((8, 8, 256)))  # Reshape into a 3D volume

    # Upsample to 16x16
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsample to 32x32
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsample to 64x64
    model.add(layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsample to 128x128 and output 3 channels
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))

    return model

#### Design of the Discriminator

In [ ]:
def build_discriminator():
    model = tf.keras.Sequential()

    # First convolutional layer: 128x128 -> 64x64
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[128, 128, 3]))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Second convolutional layer: 64x64 -> 32x32
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Third convolutional layer: 32x32 -> 16x16
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Fourth convolutional layer: 16x16 -> 8x8
    model.add(layers.Conv2D(512, (5, 5), strides=(2, 2), padding='same'))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    # Flatten and output a single unit
    model.add(layers.Flatten())
    model.add(layers.Dense(1))

    return model


#### Loss functions

In [ ]:
# Binary cross-entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)  # Generator wants fake images to be classified as real

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)  # Real images classified as real (1)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)  # Fake images classified as fake (0)
    return real_loss + fake_loss

#### Optimizers

In [ ]:
# Define learning rates for generator and discriminator
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)

#### GAN Training Loop

In [ ]:
noise_dim = 100
num_examples_to_generate = 8

# Seed for generating images during training
seed = tf.random.normal([num_examples_to_generate, noise_dim])

generator = build_generator(noise_dim)
discriminator = build_discriminator()

# Display the architecture
generator.summary()
discriminator.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 16384)               │       1,638,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 16384)               │          65,536 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 16384)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ reshape (Reshape)                    │ (None, 8, 8, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose (Conv2DTranspose)   │ (None, 16, 16, 128)         │         819,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 16, 16, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 16, 16, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_1 (Conv2DTranspose) │ (None, 32, 32, 64)          │         204,800 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 32, 32, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_2 (Conv2DTranspose) │ (None, 64, 64, 32)          │          51,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64, 64, 32)          │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_3 (LeakyReLU)            │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_transpose_3 (Conv2DTranspose) │ (None, 128, 128, 3)         │           2,400 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,782,432 (10.61 MB)

 Trainable params: 2,749,216 (10.49 MB)

...

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 64, 64, 64)          │           4,864 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_4 (LeakyReLU)            │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64, 64, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 32, 32, 128)         │         204,928 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_5 (LeakyReLU)            │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32, 32, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 16, 16, 256)         │         819,456 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_6 (LeakyReLU)            │ (None, 16, 16, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 16, 16, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 8, 8, 512)           │       3,277,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_7 (LeakyReLU)            │ (None, 8, 8, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 8, 8, 512)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │          32,769 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,339,329 (16.55 MB)

 Trainable params: 4,339,329 (16.55 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:

# Training function
@tf.function
def train_step(images, noise_dim):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        # Generate fake images
        generated_images = generator(noise, training=True)

        # Discriminator outputs for real and fake images
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        # Compute generator and discriminator losses
        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    # Compute gradients and update the generator and discriminator
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

# Function to save generated images during training
def generate_and_save_images(model, epoch, test_input, output_dir='generated_images'):
    # Generate images from the test input (fixed seed)
    predictions = model(test_input, training=False)

    # Rescale images from [-1, 1] to [0, 1] for display
    predictions = (predictions + 1) / 2.0

    # Create a grid of generated images
    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i])
        plt.axis('off')

    # Save the plot
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    plt.savefig(f'{output_dir}/image_at_epoch_{epoch:04d}.png')
    plt.show()

# Function to visualize loss curves
def plot_loss(gen_losses, disc_losses, output_dir='generated_images'):
    plt.figure()
    plt.plot(gen_losses, label='Generator Loss')
    plt.plot(disc_losses, label='Discriminator Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    plt.savefig(f'{output_dir}/loss_curve.png')
    plt.show()

# Training loop
def train(dataset, epochs, noise_dim, batch_size, save_interval=10):
    gen_losses = []
    disc_losses = []

    for epoch in range(epochs):
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch, noise_dim)

        gen_losses.append(gen_loss)
        disc_losses.append(disc_loss)

        print(f'Epoch {epoch+1}/{epochs}, Generator Loss: {gen_loss:.4f}, Discriminator Loss: {disc_loss:.4f}')

        # Save and visualize generated images every 'save_interval' epochs
        if (epoch + 1) % save_interval == 0:
            generate_and_save_images(generator, epoch + 1, seed)

    # Plot loss curves
    plot_loss(gen_losses, disc_losses)

# Define hyperparameters
batch_size = 128
epochs = 1
save_interval = 1  # Save and visualize images every 10 epochs


# Assuming 'train_dataset' is your preprocessed dataset
train(train_dataset, epochs, noise_dim, batch_size, save_interval)

Se truncaron las últimas líneas 5000 del resultado de transmisión.
Loading image: 025742.jpg
Loading image: 025743.jpg
Loading image: 025744.jpg
Loading image: 025745.jpg
Loading image: 025746.jpg
Loading image: 025747.jpg
Loading image: 025748.jpg
Loading image: 025749.jpg
Loading image: 025750.jpg
Loading image: 025751.jpg
Loading image: 025752.jpg
Loading image: 025753.jpg
Loading image: 025754.jpg
Loading image: 025755.jpg
Loading image: 025756.jpg
Loading image: 025757.jpg
Loading image: 025758.jpg
Loading image: 025759.jpg
Loading image: 025760.jpg
Loading image: 025761.jpg
Loading image: 025762.jpg
Loading image: 025763.jpg
Loading image: 025764.jpg
Loading image: 025765.jpg
Loading image: 025766.jpg
Loading image: 025767.jpg
Loading image: 025768.jpg
Loading image: 025769.jpg
Loading image: 025770.jpg
Loading image: 025771.jpg
Loading image: 025772.jpg
Loading image: 025773.jpg
Loading image: 025774.jpg
Loading image: 025775.jpg
Loading image: 025776.jpg
Loading image: 025777.j

#### Reflexión

1. ¿Qué conceptos de la teoría le resultaron más desafiantes y por qué?

Uno de los aspectos más complejos de entender en las GAN es la inestabilidad durante el entrenamiento. A menudo, se presentan problemas como el colapso de modo, donde el generador crea pocas variaciones de las imágenes, o situaciones en las que el discriminador se vuelve demasiado fuerte y domina al generador. Mantener un balance adecuado entre la tasa de aprendizaje y las capacidades del generador y el discriminador es complicado, ya que pequeños desequilibrios pueden arruinar el proceso de entrenamiento.
Otro reto es la no convergencia, ya que la competencia entre el generador y el discriminador puede hacer difícil saber si el modelo está realmente aprendiendo o simplemente pasando por ciclos de error. También es complejo entender cómo la pérdida del generador se ajusta con la retroalimentación del discriminador y por qué la minimización de la entropía cruzada binaria es eficaz para las GAN, a diferencia de otras funciones de pérdida que podrían parecer más intuitivas.


2. ¿Cómo lo ayudó el laboratorio a consolidar o comprender mejor estos conceptos?

El laboratorio brindó una experiencia práctica que facilitó la comprensión de la teoría. Por ejemplo, observar en tiempo real cómo se desequilibra el entrenamiento entre el generador y el discriminador ayudó a entender la importancia de ajustar sus actualizaciones.
Además, visualizar las imágenes generadas a lo largo de varias épocas mostró cómo las GAN aprenden progresivamente, y cómo puede surgir el colapso de modo. Esto subrayó la relevancia de afinar los hiperparámetros, como las tasas de aprendizaje o el uso del optimizador Adam con parámetros específicos.
Por otro lado, la práctica permitió concretar el proceso de preparación de datos, un aspecto que a veces se pasa por alto en la teoría. Aprender a gestionar eficientemente la carga y el preprocesamiento de datos en tiempo real es crucial para escalar las GAN a problemas del mundo real.


3. ¿Qué aplicaciones potenciales ve para las GAN en la industria o la investigación?

Las GAN tienen muchas aplicaciones prometedoras en la industria y la investigación. Por ejemplo, son útiles en la generación de imágenes y videos, aplicándose en la creación de rostros, obras de arte y texturas realistas para videojuegos.
En el ámbito médico, las GAN pueden generar imágenes sintéticas que ayudan en el diagnóstico, especialmente cuando los conjuntos de datos son limitados. También son valiosas para el aumento de datos, permitiendo mejorar el rendimiento de modelos de aprendizaje automático cuando se dispone de pocos ejemplos.
Otra aplicación es la superresolución, donde las GAN mejoran la calidad de imágenes de baja resolución, lo que resulta útil en campos como la transmisión de video o las imágenes satelitales. Por último, aunque controvertida, la tecnología de deepfakes, que utiliza GAN para generar videos falsos, tiene aplicaciones en la industria del cine y los videojuegos.

4. ¿Qué limitaciones o preocupaciones éticas puede identificar en el uso de GAN?

Existen serios problemas éticos asociados con las GAN, como la creación de deepfakes, que pueden usarse para difundir información falsa o violar la privacidad de las personas, con implicaciones sociales y políticas importantes si no se regula adecuadamente.
También hay preocupaciones sobre el sesgo en los modelos, ya que las GAN reflejan y amplifican los sesgos presentes en los datos de entrenamiento, lo que puede generar desigualdades en áreas críticas como la justicia penal o la atención médica.
Además, el entrenamiento de GAN exige una gran cantidad de recursos computacionales, lo que puede ser una barrera para quienes no tienen acceso a hardware especializado. Finalmente, el colapso de modo, donde el generador produce resultados poco variados, sigue siendo un obstáculo en su implementación.


 5. ¿Qué opinas sobre la implementación y el entrenamiento de las GAN después de la experiencia práctica?

Tras la experiencia práctica, hay una mayor confianza en la capacidad de implementar GAN funcionales, pero también queda claro que su entrenamiento es complicado y requiere un ajuste meticuloso de los hiperparámetros. Aunque la implementación parece más accesible después del laboratorio, la sensibilidad del modelo y la inestabilidad del entrenamiento muestran que es necesario un enfoque iterativo y cuidadoso para obtener buenos resultados.
También se destacó la importancia del manejo adecuado de los datos, y cómo una canalización eficiente puede mejorar tanto la velocidad como el uso de la memoria. Finalmente, la experiencia ayudó a comprender los problemas más comunes que pueden surgir, como la desaparición de gradientes, el colapso de modo y la falta de convergencia.

 6. Conclusión:

La experiencia práctica ayudó a clarificar muchos de los desafíos teóricos de las GAN, como la inestabilidad del entrenamiento y el comportamiento de las funciones de pérdida. El laboratorio también facilitó la comprensión de cómo las GAN se comportan en la práctica, destacando sus amplias aplicaciones, pero también exponiendo las limitaciones y preocupaciones éticas que deben considerarse en su uso.
